In [1]:
import modulos as md

symbol='COMBOUSDT'
data = md.obtiene_historial(symbol)
data = md.estrategia_bb(data)
resultado = md.backtesting(data)
resultado






C:\Users\Luis Gomez Freites\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Start                     2023-06-09 01:00:00
End                       2023-06-29 20:30:00
Duration                     20 days 19:30:00
Exposure Time [%]                        30.3
Equity Final [$]                  1198.074367
Equity Peak [$]                   1207.074367
Return [%]                          19.807437
Buy & Hold Return [%]               -28.86143
Return (Ann.) [%]                 2212.718534
Volatility (Ann.) [%]             1477.442059
Sharpe Ratio                         1.497669
Sortino Ratio                      291.918612
Calmar Ratio                       352.829293
Max. Drawdown [%]                   -6.271357
Avg. Drawdown [%]                   -1.322539
Max. Drawdown Duration       12 days 13:30:00
Avg. Drawdown Duration        1 days 00:38:00
# Trades                                    3
Win Rate [%]                        66.666667
Best Trade [%]                      15.805785
Worst Trade [%]                     -2.013806
Avg. Trade [%]                    